UK MONETARY POLICY 2019-2025

## Objectives

* The objectives of this project are to:

Collect official UK macroeconomic data from reliable public sources

Apply Python-based ETL (Extract, Transform, Load) techniques

Clean and preprocess time-series data for analysis

Convert datasets to a common quarterly frequency for alignment

Analyse trends in inflation, Official Bank Rate and GDP between 2019 and 2025
## Inputs

* The following data inputs required: 
1. Bank of England Policy Interest Rate : time series data on the UK nominal policy interest rate 
2. Consumer Price Index(CPI): UK CPI, used as a measure inflation
3. Gross Domestic Product(GDP) : UK GDP data, used to represent economic activity. GDP

## Outputs

* Write here which files, code or artefacts you generate by the end of the notebook 

## Additional Comments

* If you have any additional comments that don't fit in the previous bullets, please state them here. 



---

# Change working directory

* We are assuming you will store the notebooks in a subfolder, therefore when running the notebook in the editor, you will need to change the working directory

We need to change the working directory from its current folder to its parent folder
* We access the current directory with os.getcwd()

In [ ]:
import os
current_dir = os.getcwd() #
current_dir

We want to make the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

In [ ]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

Confirm the new current directory

In [ ]:
current_dir = os.getcwd()
current_dir

# Section 1
#Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 

Section 1 Extraction: Load the Dataset

In [ ]:
import pandas as pd

cpi = pd.read_csv(r"\\talktalk\redirectedfolders\F.Afolabi\Documents\VSCode1\inflation.csv")
print(cpi.head())

# Display basic information about the dataset, I would take the variable one by one; manipulating CPI first; 
cpi.info() 

print(cpi.shape)  # Looking at the shape of CPI

In [ ]:
# Generate a summary of the statistics
print(cpi.describe())

In [ ]:
# Checking for missing values and data type
print(cpi.isnull().sum())
print(cpi.dtypes)

In [ ]:
# Converting Date to datetime
cpi['Date'] = pd.to_datetime(cpi['Date'], errors='coerce')

In [ ]:
# Filtering the data between 2019 and 2025 for CPI for the analysis and the scope of this project
cpi = cpi[(cpi['Date'] >= '2019-01-01') & (cpi['Date'] <= '2025-12-31')]
print(cpi.head())
cpi.info()

In [ ]:
#filling missing values using interpolation method
cpi['CPI_Inflation'] = cpi['CPI_Inflation'].interpolate()


Converting CPI From Annual to Quarterly Frequency

In [ ]:
#Annual aggregation of CPI Inflation
cpi['Year'] = cpi['Date'].dt.year
cpi_annual = (cpi.groupby('Year', as_index=False).agg({'CPI_Inflation': 'mean'}))
print(cpi_annual)

In [ ]:
#Converting annual CPI to quarterly CPI by forward filling the values

q_index = pd.period_range('2019Q1', '2025Q4', freq='Q')

cpi_q = cpi_annual.copy()

# Creating a  Date column from Year
cpi_q['Date'] = pd.PeriodIndex(cpi_q['Year'].astype(str) + 'Q1', freq='Q')

cpi_q = cpi_q[['Date', 'CPI_Inflation']] # Selecting only the necessary columns



#Reindexing and forward filling
cpi_q = (cpi_q.set_index('Date').reindex(q_index).ffill().reset_index().rename(columns={'index': 'Date'}))









In [ ]:
print(len(cpi_q))
print(cpi_q.head())
print(cpi_q.tail())

In [ ]:
cpi_q.info()

In [ ]:
 cpi_q_cleaned = cpi_q.to_csv('cpi_q_cleaned.csv', index=False) # Saving the cleaned CPI data to a new CSV file, for further analysis

Loading the Official Bank Rate to be cleaned. 

In [ ]:
boe_rate = pd.read_csv(r"\\talktalk\redirectedfolders\F.Afolabi\Documents\VSCode1\Bank_rate.csv")
print(boe_rate.head())

In [ ]:
#boe_rate = pd.read_csv('../Dataset/Raw/Bank_rate.csv')
#boe_rate.head()
    

In [ ]:
boe_rate.info() #Looking at the data types and missing values

In [ ]:
# Rename the Colum, I rename the column to be consistent across the dataset

boe_rate.columns = ['Date', 'Bank_Rate']

In [ ]:
boe_rate.info()

In [ ]:
print(boe_rate.columns)

In [ ]:
# Convert Date column safely (UK date format)
boe_rate['Date'] = pd.to_datetime(
    boe_rate['Date'],
    errors='coerce',
    dayfirst=True
)

# Remove rows with invalid dates
boe_rate = boe_rate.dropna(subset=['Date'])

# Set Date as index for resampling
boe_rate = boe_rate.set_index('Date')

# Resample to quarterly frequency (mean Bank Rate)
boe_rate_q = boe_rate[['Bank_Rate']].resample('QS').mean()

print(boe_rate_q.head())

In [ ]:
# Filtering the data between 2019 and 2025 for Bank Rate  for the analysis and the scope of this project, date reset to allow filtering
boe_rate.reset_index(inplace=True)
boe_rate_filtered = boe_rate[(boe_rate['Date'] >= '2019-01-01') & (boe_rate['Date'] <= '2025-12-31')]
print(boe_rate_filtered.head())
boe_rate_filtered.info()

In [ ]:
boe_rate_filtered.dtypes

In [ ]:
boe_rate_filtered.info()


In [ ]:
# Checking for missing values
print(boe_rate_filtered.isnull().sum())

In [ ]:
#Checking for missing Quaterly periods
#boe_rate_filtered = boe_rate_filtered.asfreq('QS')
#print(boe_rate_filtered.isnull().sum())


In [ ]:
#print Bank Rtae info
boe_rate_filtered.info()

In [ ]:
# Resetting index

# boe_rate_filtered=boe_rate_filtered.reset_index() 

In [ ]:
#Dropping the index column because it is not needed
#boe_rate_filtered = boe_rate_filtered.drop(columns=['index'])

In [ ]:
# Convert Date to Period for consistency
#boe_rate_filtered['Date'] = boe_rate_filtered['Date'].dt.to_period('Q')


In [ ]:
print(boe_rate_filtered['Date'].dtype)

In [ ]:
# saved the cleaned Bank Rate data to a new CSV file, for further analysis
boe_rate_cleaned = boe_rate_filtered.to_csv('boe_rate_cleaned.csv', index=True)

In [ ]:
print(boe_rate_filtered.head())

In [ ]:
# Loading GDP data
gdp = pd.read_csv('../Dataset/Raw/GDP.csv')
gdp.head(5)


In [ ]:
print(gdp.info)

In [ ]:
print(gdp.columns)

In [ ]:
#Finding all columns containing 'Gross' or 'GDP' becasue they are relevant to our analysis 
gdp_columns = [col for col in gdp.columns if 'Gross' in col or 'GDP' in col]
print(gdp_columns)

In [ ]:
for col in gdp.columns:
    print(repr(col))

In [ ]:
gdp.columns = (
    gdp.columns
      .astype(str)
      .str.encode('latin1', errors='ignore').str.decode('utf-8', errors='ignore')
      .str.replace(r'\s+', ' ', regex=True)
      .str.replace('\\', '', regex=False)
      .str.strip()
)

In [ ]:
print(list(gdp.columns))


In [ ]:
for i, col in enumerate(gdp.columns):
    print(i, repr(col))

In [ ]:
# SLICING: I need to create a new dataframe with only the relevant columns

gdp_clean = gdp[['Date', 'Gross Domestic Product']].copy() 

# Rename columns for clarity
gdp_clean = gdp_clean.rename(columns={'Gross Domestic Product': 'GDP_Level'})

In [ ]:
gdp_clean.head()
gdp_clean.info()


In [ ]:
#Converting 'Date' to string to enable filtering



#converting GDP to numeric
gdp_clean['GDP_Level'] = pd.to_numeric(gdp_clean['GDP_Level'], errors='coerce')






In [ ]:
# filtering for the period 2019 Q1 to 2025 Q4
gdp_clean = gdp_clean[(gdp_clean['Date'] >= '2019Q1') & (gdp_clean['Date'] <= '2025Q4')].copy()


In [ ]:
gdp_clean.info()
gdp_clean.head()
gdp_clean.tail()

In [ ]:
print(gdp_clean['Date'].min(), gdp_clean['Date'].max())
print(gdp_clean['Date'].dtype)

GDP dates were converted to quarterly period format for alignment with both Bank rate and Inflation(CPI)

In [ ]:
gdp_clean_cleaned = gdp_clean.to_csv('gdp_cleaned.csv', index=False) # Saving the cleaned GDP data to a new CSV file, for further analysis
gdp_clean_cleaned

In [ ]:
print(gdp_clean['Date'].dtype)
print(boe_rate_filtered['Date'].dtype)
print(cpi_q['Date'].dtype)

In [ ]:
final_df = (
    gdp_clean
    .merge(boe_rate_filtered, on='Date', how='left')
    .merge(cpi_q, on='Date', how='left')
)

In [ ]:
print(final_df.head())
print(final_df.info())

In [ ]:
print("GDP dates:")
print(gdp_clean['Date'].min(), gdp_clean['Date'].max())

print("Bank Rate dates:")
print(boe_rate_filtered['Date'].min(), boe_rate_filtered['Date'].max())

print("CPI dates:")
print(cpi_q['Date'].min(), cpi_q['Date'].max())

In [ ]:
# save the final df to csv file
final_df.to_csv("final_df", index=False)


---

# Section 2

Section 2 content

---

NOTE

* You may add as many sections as you want, as long as it supports your project workflow.
* All notebook's cells should be run top-down (you can't create a dynamic wherein a given point you need to go back to a previous cell to execute some task, like go back to a previous cell and refresh a variable content)

---

# Push files to Repo

* In cases where you don't need to push files to Repo, you may replace this section with "Conclusions and Next Steps" and state your conclusions and next steps.

In [ ]:
import os
#try:
  # create your folder here
  # os.makedirs(name='')
#except Exception as e:
 # print(e)
